In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.environ['KAGGLE_USERNAME'] = "lampe01" # username from the json file
os.environ['KAGGLE_KEY'] = "0fe21bb583c2cd3230e9d928c96e27f7" # key from the json file
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/CirCor"

In [3]:
pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
pip install Ipython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
pip uninstall scikit-learn

Found existing installation: scikit-learn 0.24.2
Uninstalling scikit-learn-0.24.2:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/scikit_learn-0.24.2.dist-info/*
    /usr/local/lib/python3.7/dist-packages/scikit_learn.libs/libgomp-f7e03b3e.so.1.0.0
    /usr/local/lib/python3.7/dist-packages/sklearn/*
Proceed (y/n)? y
  Successfully uninstalled scikit-learn-0.24.2


In [7]:
pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scikit_learn-1.0.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (24.8 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyaudioprocessing 1.1.13 requires scikit-learn==0.24.2; python_version <= "3.7", but you have scikit-learn 1.0.2 which is incompatible.


In [8]:
pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
pip install pyAudioProcessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scikit_learn-0.24.2-cp37-cp37m-manylinux2010_x86_64.whl (22.3 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.5 requires scikit-learn>=1.0.0, but you have scikit-learn 0.24.2 which is incompatible.


In [10]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
import itertools
import os
import sys
import pywt
import librosa
import sklearn
import scipy
import scipy.io
import scipy.io.wavfile
from scipy import signal
from scipy.stats import kurtosis, skew, tstd
from scipy.signal import periodogram, welch, butter, lfilter, freqz
from os import path
from pydub import AudioSegment
import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras.models import Model, load_model, Sequential
from keras.layers import Input, Dense, Dropout, AlphaDropout, Activation, Flatten, LSTM
from keras.callbacks import ModelCheckpoint, TensorBoard,EarlyStopping,ReduceLROnPlateau,ProgbarLogger
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, precision_recall_curve, recall_score, classification_report, auc, roc_curve, precision_recall_fscore_support, f1_score, accuracy_score
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from pyAudioProcessing.extract_features import get_features

In [11]:
mdata = pd.read_csv('/content/drive/MyDrive/ReCirCor/training_data.csv')
input_dir = '/content/drive/MyDrive/ReCirCor/training_data/training_data'

In [12]:
#Gets all the txt files
def getfilenames(inputdir):
    Fname = list()
    Fname = sorted([fname for fname in os.listdir(inputdir) if not fname.startswith('.')and fname.endswith('.txt')])
    
    return Fname

In [13]:
#List of Labels - 0 for normal, 1 for murmur
labels = [0,0,0,0,0,1,1,1,1,1,1,1,1,0,1,1,1,0,0,0,1,1,1]
len(labels)

23

In [14]:
#They were originally named differently and I have renamed and numbered them for ease of access.
#The michigan heart sound recordings are in mp3 format
#The recordings are converted to wav format for easy processing as most python libraries require the audio format to be wav.

def converttowav(filepath):
  destinationpath = filepath.split('.')[0] + '.wav'
  sound = AudioSegment.from_mp3(filepath)
  sound.export(destinationpath, format = "wav")

filepathslist = list()
for i in range(23):
  filepath = str(i+1) + '.mp3'
  converttowav(filepath)
  filepathslist.append(str(i+1)+'.wav')

In [15]:
#Creating an empty dataframe with columns to store filenames and the respective labels
dataframe = pd.DataFrame(columns = ['filename', 'labels'])

In [16]:
filepathslist

['1.wav',
 '2.wav',
 '3.wav',
 '4.wav',
 '5.wav',
 '6.wav',
 '7.wav',
 '8.wav',
 '9.wav',
 '10.wav',
 '11.wav',
 '12.wav',
 '13.wav',
 '14.wav',
 '15.wav',
 '16.wav',
 '17.wav',
 '18.wav',
 '19.wav',
 '20.wav',
 '21.wav',
 '22.wav',
 '23.wav']

In [17]:
labels

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1]

In [18]:
labels[18]

0

In [19]:
#Inserting filename and label values into dataframe
dataframe['filename'] = filepathslist
dataframe['labels'] = labels

In [20]:
dataframenew = pd.DataFrame(columns = ['new ID', 'wav filename', 'labels'])

In [21]:
import random

In [22]:
newID = list()
wavfilename = list()
newlabels = list()
for i in range(250):
  x = random.randint(1,23)
  newID.append(str(i+1))
  wavfilename.append(str(x)+'.wav')
  newlabels.append(labels[x-1])

In [23]:
#Get only murmur recordings
def getrecmur(inputdir):
    fname = getfilenames(inputdir)
    items = []
    rinfo = []
    recf = []
    labels = []
    recfile = []
    for i in fname:
        i = inputdir + '/' + i
        with open(i, 'r') as f:
            content = f.readlines()
        numloc = int(content[0].split(' ')[1])
        for i in range(numloc):
            for p in range(len(content)):
                t = content[p].split('\n')
                for it in t:
                    if it.startswith('#Murmur:'):
                        tar = it.split(': ')[1]
                        if tar == 'Present':
                          recinfo = content[i+1].split(' ')
                          rinfo.append(recinfo)
                          l = 1
                          labels.append(l)
                        elif tar == 'Absent':
                          recinfo = content[i+1].split(' ')
                          rinfo.append(recinfo)
                          l = 0
                          labels.append(l)
    for items in rinfo:
        for i in range(len(items)):
            if items[i].endswith('.wav'):
                recfile = input_dir + '/' + items[i]
        recf.append(recfile)
    return rinfo, recf, labels

rin, ref, label = getrecmur(input_dir)

In [24]:
len(rin)

3007

In [25]:
ref

['/content/drive/MyDrive/ReCirCor/training_data/training_data/13918_AV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/13918_PV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/13918_TV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/13918_MV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/14241_AV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/14241_PV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/14241_TV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/14241_MV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/14998_AV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/14998_PV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/14998_TV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/14998_MV.wav',
 '/content/drive/MyDrive/ReCirCor/training_data/training_data/23625_AV.wav',

In [26]:
x = 250
for i in range(100):
  x = x + 1
  wavfilename.append(ref[i])
  newlabels.append(label[i])
  newID.append(x)

In [27]:
len(newID)

350

In [28]:
dataframenew['new ID'] = newID
dataframenew['wav filename'] = wavfilename
dataframenew['labels'] = newlabels
dataframenew

,new ID,wav filename,labels
0,1,3.wav,0
1,2,10.wav,1
2,3,11.wav,1
3,4,19.wav,0
4,5,18.wav,0
...,...,...,...
345,346,/content/drive/MyDrive/ReCirCor/training_data/...,0
346,347,/content/drive/MyDrive/ReCirCor/training_data/...,0
347,348,/content/drive/MyDrive/ReCirCor/training_data/...,0
348,349,/content/drive/MyDrive/ReCirCor/training_data/...,1


In [29]:
dataframe

,filename,labels
0,1.wav,0
1,2.wav,0
2,3.wav,0
3,4.wav,0
4,5.wav,0
5,6.wav,1
6,7.wav,1
7,8.wav,1
8,9.wav,1
9,10.wav,1


In [30]:
dataframenew['labels'].value_counts()

1    198
0    152
Name: labels, dtype: int64

In [31]:
X_train, X_test, y_Train, y_Test = train_test_split(dataframenew['new ID'], dataframenew['labels'], train_size=0.8, shuffle=True)

In [32]:
X_train

236    237
94      95
226    227
271    272
164    165
      ... 
163    164
233    234
123    124
208    209
78      79
Name: new ID, Length: 280, dtype: object

In [33]:
X_test

302    303
36      37
231    232
166    167
116    117
      ... 
103    104
189    190
44      45
187    188
138    139
Name: new ID, Length: 70, dtype: object

In [34]:
#Computing Features - 

#Discrete Wavelet Transform
def computeDWTs(x):
    #approximation and detail coefficients
    res = np.array([])
    ca, cd = pywt.dwt(x, 'db8')
    #Compute mean, stdev, skew, kurtosis
    camean = np.mean(ca)
    res = np.hstack((res,camean))
    castdev = tstd(ca)
    res = np.hstack((res,castdev))
    caskew = skew(ca)
    res = np.hstack((res,caskew))
    cakurtosis = kurtosis(ca)
    res = np.hstack((res,cakurtosis))
    cdmean = np.mean(cd)
    res = np.hstack((res,cdmean))
    cdstdev = tstd(cd)
    res = np.hstack((res,cdstdev))
    cdskew = skew(cd)
    res = np.hstack((res,cdskew))
    cdkurtosis = kurtosis(cd)
    res = np.hstack((res, cdkurtosis))
    return res

#Computing the Spectral Entropy of the signal - Shannon Entropy of the Power Spectral Density (PSD) of the data
def SpectralEntropy(Signal, SampleRate, method = 'fft', nperseg = None, normalize = False):
    Signal = np.array(Signal)
    if method == 'fft':
      _, psd = periodogram(Signal, SampleRate)
    elif method == 'welch':
      _, psd = welch(Signal, SampleRate, nperseg=nperseg)
    psd_norm = np.divide(psd, psd.sum())
    se = -np.multiply(psd_norm, np.log2(psd_norm)).sum()
    if normalize:
        se /= np.log2(psd_norm.size)
    return se

#Computing the Power Spectral Density(PSD) of signal - 
def computepsd(x):
    freq, psd = signal.welch(x)
    return psd

#Computing Mel-frequency cepstral coefficients, Gammatone frequency cepstral coefficients, RMS Energy, Zero-crossing rate, mean, variance, skew, and kurtosis of the signal
def computeMFCC(x, sr):
    mfccs = np.mean(librosa.feature.mfcc(y = x, sr = sr, n_mfcc = 13).T, axis = 0)
    return mfccs

def computeGFCC(file_name):
    gfcc = get_features(file=file_name, feature_names=["gfcc"])
    gfcc = np.array(gfcc['audio'][file_name]['features'])
    return gfcc

def computeRMSEn(x):
    rmsen = np.mean(librosa.feature.rms(x, frame_length=2048, hop_length=1024, center=True).T, axis = 0)
    return rmsen

def computeZCR(x):
    zcr = np.mean(librosa.feature.zero_crossing_rate(y = x, frame_length=2048, hop_length=1024).T, axis = 0)
    return zcr

def computestats(x):
#Mean, Variance, Skew, Kurtosis
    stats = []
    me = np.mean(x)
    stats.append(me)
    variance = np.var(x)
    stats.append(variance)
    sk = skew(x)
    stats.append(sk)
    kurt = kurtosis(x)
    stats.append(kurt)
    return stats

In [35]:
#Function to extract the required features
def extractfeatures(file_name, x, sr, *, mfcc: bool = False, dwt: bool = False, spectral_entropy: bool = False, psd: bool = False, gfcc: bool = False, rmse: bool = False, zcr: bool = False, stats: bool = False):
    finalfeatures = np.array([])
    if mfcc:
        mfccs = computeMFCC(x, sr)
        
        finalfeatures = np.hstack((finalfeatures, mfccs))
    if dwt:
        dwtr = computeDWTs(x)
        finalfeatures = np.hstack((finalfeatures, dwtr))
    if spectral_entropy:
        specen = SpectralEntropy(x, sr)
        finalfeatures = np.hstack((finalfeatures,specen))
    if psd:
        psds = computepsd(x)
        finalfeatures = np.hstack((finalfeatures, psds))
    if gfcc:
        gfccs = computeGFCC(file_name)
        finalfeatures = np.hstack((finalfeatures, gfccs))
    if rmse:
        rmses = computeRMSEn(x)
        finalfeatures = np.hstack((finalfeatures, rmses))
    if zcr:
        zcrs = computeZCR(x)
        finalfeatures = np.hstack((finalfeatures, zcrs))
    if stats:
        stats1 = computestats(x)
        finalfeatures = np.hstack((finalfeatures, stats1))
    return finalfeatures

In [36]:
#loads audio and extracts required features
def load_file_data(file_num, duration=15, sr = 4000):
    input_length=sr*duration
    notfeatures = []
    i = 0
    for file_no in file_num:
        try:
            #sound_file = folder + '/' + file_name
            sfile = int(file_no)
            sound_file = dataframenew['wav filename'][sfile-1]
            print ("load file ", i, sound_file)
            x, sr = librosa.load(sound_file,duration=duration,res_type='kaiser_fast')
            dur = librosa.get_duration(y=x, sr=sr)
            #Padding audio signal
            if (round(dur) < duration):
                print ("fixing audio length:", file_no, sound_file)
                y = librosa.util.fix_length(x, input_length)
            feature = extractfeatures(sound_file, x, sr, mfcc = True, dwt = True, psd = True, gfcc = True, rmse = True, zcr = True, stats = True)
            #dwt = True, psd = True, gfcc = True, rmse = True, zcr = True, stats = True
            notfeatures.append(feature)          
        except Exception as e:
            print(e)
            #notfeatures = np.asarray(features, dtype=np.float32)
        i = i+1 
    return notfeatures

In [37]:
x_train = []
x_test = []
x_train = load_file_data(X_train)
x_test = load_file_data(X_test)

load file  0 2.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : 2.wav
load file  1 2.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : 2.wav
load file  2 19.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : 19.wav
load file  3 /content/drive/MyDrive/ReCirCor/training_data/training_data/2530_PV.wav
fixing audio length: 272 /content/drive/MyDrive/ReCirCor/training_data/training_data/2530_PV.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : /content/drive/MyDrive/ReCirCor/training_data/training_data/2530_PV.wav
load file  4 19.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : 19.wav
load file  5 18.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : 18.wav
load file  6 3.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : 3.wav
lo

In [38]:
x_train = np.asarray(x_train).astype(np.float32)
x_test = np.asarray(x_test).astype(np.float32)

In [39]:
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [40]:
x_test.shape

(70, 200)

In [41]:
x_train.shape

(280, 200)

In [42]:
y_Train.dtype

dtype('int64')

In [43]:
y_Train = np.asarray(y_Train).astype(np.float32)

In [44]:
y_Test = np.asarray(y_Test).astype(np.float32)

In [45]:
traindataframe = pd.DataFrame(columns = ['x_train ID', 'labels'])
testdataframe = pd.DataFrame(columns = ['x_test ID', 'labels'])
traindataframe['x_train ID'] = X_train
testdataframe['x_test ID'] = X_test
traindataframe['labels'] = y_Train
testdataframe['labels'] = y_Test

In [46]:
testdataframe

,x_test ID,labels
302,303,0.0
36,37,0.0
231,232,0.0
166,167,1.0
116,117,1.0
...,...,...
103,104,0.0
189,190,1.0
44,45,1.0
187,188,1.0


In [47]:
xtrainwavfiles = list()
for i in X_test:
  num = int(i)
  xtrainwavfiles.append(dataframenew['wav filename'][num-1])

In [48]:
pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [49]:
import kerastuner
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
def model_builder(hp):
  model = Sequential()
  hp_units1 = hp.Int('units_', min_value=2, max_value=64, step=4)
  model.add(tf.keras.layers.Dense(units = hp_units1, input_dim = x_train.shape[1]))

  # Tune the number of dense layers
  for i in range(hp.Int('num_layers', 1, 5)):
    
    # Tune the number of units in the each dense layer
    hp_units = hp.Int('units_'+str(i), min_value=2, max_value=64, step=4)
    model.add(tf.keras.layers.Dense(units=hp_units, activation='relu'))
    
    # Tune the dropout rate in the each dense layer
    hp_dropout = hp.Float('rate', min_value=0.0, max_value=0.5, step=0.1)
    model.add(tf.keras.layers.Dropout(hp_dropout))
    
  # Add dense output layer
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

  # Tune the learning rate for the optimizer
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),loss='binary_crossentropy',
                metrics=['accuracy'])

  return model

#tf.keras.metrics.Recall()
#tuner = BayesianOptimization(model_builder,
#max_trials=50,
#objective='val_accuracy',
#overwrite=True)

tuner = Hyperband(model_builder,
objective=kerastuner.Objective("val_accuracy", direction="max"),
max_epochs=50,
factor=3, overwrite = True)

tuner.search(x_train,y_Train, epochs=50, validation_split=0.2)
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

model = tuner.hypermodel.build(best_hps)

Trial 90 Complete [00h 00m 04s]
val_accuracy: 0.9285714030265808

Best val_accuracy So Far: 0.9285714030265808
Total elapsed time: 00h 03m 07s


In [50]:
history = model.fit(x_train, y_Train, batch_size = 10, epochs=50, validation_split = 0.2, verbose = 1)

Epoch 1/50
23/23 [==============================] - 1s 9ms/step - loss: 0.6486 - accuracy: 0.5089 - val_loss: 0.6677 - val_accuracy: 0.5714
Epoch 2/50
23/23 [==============================] - 0s 3ms/step - loss: 0.6363 - accuracy: 0.5804 - val_loss: 0.6719 - val_accuracy: 0.4643
Epoch 3/50
23/23 [==============================] - 0s 3ms/step - loss: 0.6390 - accuracy: 0.5804 - val_loss: 0.6759 - val_accuracy: 0.4643
Epoch 4/50
23/23 [==============================] - 0s 4ms/step - loss: 0.6353 - accuracy: 0.5804 - val_loss: 0.6785 - val_accuracy: 0.4643
Epoch 5/50
23/23 [==============================] - 0s 5ms/step - loss: 0.6344 - accuracy: 0.5804 - val_loss: 0.6766 - val_accuracy: 0.4643
Epoch 6/50
23/23 [==============================] - 0s 5ms/step - loss: 0.6372 - accuracy: 0.5804 - val_loss: 0.6776 - val_accuracy: 0.4643
Epoch 7/50
23/23 [==============================] - 0s 5ms/step - loss: 0.6369 - accuracy: 0.5804 - val_loss: 0.6794 - val_accuracy: 0.4643
Epoch 8/50
23/23 [==

In [51]:
y_predict = np.argmax(model.predict(x_test), axis=-1)
y_pred = model.predict(x_test)
y_pred=np.argmax(y_pred, axis=1)

3/3 [==============================] - 0s 4ms/step


In [52]:
print("Classification Report : \n\n", classification_report(y_Test, y_pred))

Classification Report : 

               precision    recall  f1-score   support

         0.0       0.40      1.00      0.57        28
         1.0       0.00      0.00      0.00        42

    accuracy                           0.40        70
   macro avg       0.20      0.50      0.29        70
weighted avg       0.16      0.40      0.23        70



In [53]:
df = pd.DataFrame({'Input':X_test, 'Wav filename':xtrainwavfiles, 'Real Values':y_Test, 'Predicted Values':y_pred})
df

,Input,Wav filename,Real Values,Predicted Values
302,303,/content/drive/MyDrive/ReCirCor/training_data/...,0.0,0
36,37,18.wav,0.0,0
231,232,5.wav,0.0,0
166,167,23.wav,1.0,0
116,117,8.wav,1.0,0
...,...,...,...,...
103,104,1.wav,0.0,0
189,190,7.wav,1.0,0
44,45,9.wav,1.0,0
187,188,13.wav,1.0,0


In [54]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
clf = AdaBoostClassifier(n_estimators=100, random_state=0)
clf.fit(x_train, y_Train)

AdaBoostClassifier(n_estimators=100, random_state=0)

In [55]:
clf.score(x_test, y_Test)

0.9285714285714286

In [56]:
y_pred1 = clf.predict(x_test)
print("Classification Report : \n\n", classification_report(y_Test, y_pred1))

Classification Report : 

               precision    recall  f1-score   support

         0.0       0.93      0.89      0.91        28
         1.0       0.93      0.95      0.94        42

    accuracy                           0.93        70
   macro avg       0.93      0.92      0.93        70
weighted avg       0.93      0.93      0.93        70



In [57]:
df1 = pd.DataFrame({'Input':X_test, 'Wav filename':xtrainwavfiles, 'Real Values':y_Test, 'Predicted Values':y_pred1})
df1

,Input,Wav filename,Real Values,Predicted Values
302,303,/content/drive/MyDrive/ReCirCor/training_data/...,0.0,0.0
36,37,18.wav,0.0,0.0
231,232,5.wav,0.0,0.0
166,167,23.wav,1.0,1.0
116,117,8.wav,1.0,1.0
...,...,...,...,...
103,104,1.wav,0.0,0.0
189,190,7.wav,1.0,1.0
44,45,9.wav,1.0,1.0
187,188,13.wav,1.0,1.0


In [58]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

In [59]:
clf2 = LinearSVC(penalty='l2', loss='hinge')
clf2.fit(x_train, y_Train)
clf2.score(x_test, y_Test)

0.8571428571428571

In [60]:
y_pred2 = clf2.predict(x_test)
print("Classification Report : \n\n", classification_report(y_Test, y_pred2))

Classification Report : 

               precision    recall  f1-score   support

         0.0       0.82      0.82      0.82        28
         1.0       0.88      0.88      0.88        42

    accuracy                           0.86        70
   macro avg       0.85      0.85      0.85        70
weighted avg       0.86      0.86      0.86        70



In [61]:
df2 = pd.DataFrame({'Input':X_test, 'Wav filename':xtrainwavfiles, 'Real Values':y_Test, 'Predicted Values':y_pred2})
df2

,Input,Wav filename,Real Values,Predicted Values
302,303,/content/drive/MyDrive/ReCirCor/training_data/...,0.0,0.0
36,37,18.wav,0.0,0.0
231,232,5.wav,0.0,0.0
166,167,23.wav,1.0,1.0
116,117,8.wav,1.0,1.0
...,...,...,...,...
103,104,1.wav,0.0,0.0
189,190,7.wav,1.0,1.0
44,45,9.wav,1.0,1.0
187,188,13.wav,1.0,1.0


In [62]:
clf4 = SGDClassifier(loss = 'hinge').fit(x_train, y_Train)

In [63]:
clf4.score(x_test, y_Test)

0.8571428571428571

In [64]:
y_pred4 = clf4.predict(x_test)
#y_pred=np.argmax(y_pred, axis=1)
print("Classification Report : \n\n", classification_report(y_Test, y_pred4))

Classification Report : 

               precision    recall  f1-score   support

         0.0       0.82      0.82      0.82        28
         1.0       0.88      0.88      0.88        42

    accuracy                           0.86        70
   macro avg       0.85      0.85      0.85        70
weighted avg       0.86      0.86      0.86        70



In [65]:
df4 = pd.DataFrame({'Input':X_test, 'Wav filename':xtrainwavfiles, 'Real Values':y_Test, 'Predicted Values':y_pred4})
df4

,Input,Wav filename,Real Values,Predicted Values
302,303,/content/drive/MyDrive/ReCirCor/training_data/...,0.0,1.0
36,37,18.wav,0.0,0.0
231,232,5.wav,0.0,0.0
166,167,23.wav,1.0,1.0
116,117,8.wav,1.0,1.0
...,...,...,...,...
103,104,1.wav,0.0,0.0
189,190,7.wav,1.0,1.0
44,45,9.wav,1.0,1.0
187,188,13.wav,1.0,1.0


In [66]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, f_classif, chi2

In [67]:
anova_filter = SelectKBest(f_classif, k=50)
clf = LinearSVC()
anova_svm = make_pipeline(anova_filter, clf)
anova_svm.fit(x_train, y_Train)

Pipeline(steps=[('selectkbest', SelectKBest(k=50)), ('linearsvc', LinearSVC())])

In [68]:
y_pred = anova_svm.predict(x_test)
print(classification_report(y_Test, y_pred))

              precision    recall  f1-score   support

         0.0       0.85      0.82      0.84        28
         1.0       0.88      0.90      0.89        42

    accuracy                           0.87        70
   macro avg       0.87      0.86      0.87        70
weighted avg       0.87      0.87      0.87        70



In [69]:
chi2_filter = SelectKBest(chi2, k=50)
clf = LinearSVC()
chi2_svm = make_pipeline(chi2_filter, clf)
xnew = chi2_filter.fit_transform(x_train, y_Train)

In [73]:
chi2_svm.fit(x_train, y_Train)

Pipeline(steps=[('selectkbest',
                 SelectKBest(k=50,
                             score_func=<function chi2 at 0x7fa0f30de680>)),
                ('linearsvc', LinearSVC())])

In [74]:
chi2_svm.score(x_test, y_Test)

0.6

In [76]:
y_pred = chi2_svm.predict(x_test)
print(classification_report(y_Test, y_pred))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        28
         1.0       0.60      1.00      0.75        42

    accuracy                           0.60        70
   macro avg       0.30      0.50      0.37        70
weighted avg       0.36      0.60      0.45        70



In [81]:
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA, NMF
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

In [90]:
pipe_steps = [('scaler', MinMaxScaler()), ('pca', PCA()), ('SVM', SVC(kernel='rbf'))]

check_params= {
    'pca__n_components': [4], 
    'SVM__C': [0.1, 0.5, 1, 10,30, 40, 50, 75, 100, 500, 1000], 
    'SVM__gamma' : [0.01, 0.05, 0.07, 0.1, 0.5, 1, 5, 10, 50]
}

pipeline = Pipeline(pipe_steps)

In [91]:
from tqdm import tqdm_notebook as tqdm 
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings("ignore")



for cv in tqdm(range(4,11)):
    create_grid = GridSearchCV(pipeline, param_grid=check_params, cv=cv)
    create_grid.fit(x_train, y_Train)
    print("score for %d fold CV -  %3.2f" %(cv, create_grid.score(x_test, y_Test)))
    print(" ")
    print(" ")
    print (create_grid.best_params_)

  0%|          | 0/7 [00:00<?, ?it/s]

score for 4 fold CV -  0.84
 
 
{'SVM__C': 10, 'SVM__gamma': 10, 'pca__n_components': 4}
score for 5 fold CV -  0.81
 
 
{'SVM__C': 50, 'SVM__gamma': 5, 'pca__n_components': 4}
score for 6 fold CV -  0.86
 
 
{'SVM__C': 1, 'SVM__gamma': 10, 'pca__n_components': 4}
score for 7 fold CV -  0.83
 
 
{'SVM__C': 30, 'SVM__gamma': 10, 'pca__n_components': 4}
score for 8 fold CV -  0.83
 
 
{'SVM__C': 75, 'SVM__gamma': 5, 'pca__n_components': 4}
score for 9 fold CV -  0.86
 
 
{'SVM__C': 1, 'SVM__gamma': 10, 'pca__n_components': 4}
score for 10 fold CV -  0.83
 
 
{'SVM__C': 30, 'SVM__gamma': 5, 'pca__n_components': 4}
